# Agenci Azure AI z obsługą Model Context Protocol (MCP) - Python

Ten notebook pokazuje, jak korzystać z agentów Azure AI z narzędziami Model Context Protocol (MCP) w Pythonie. Demonstruje, jak stworzyć inteligentnego agenta, który może wykorzystywać zewnętrzne serwery MCP (takie jak Microsoft Learn) dla rozszerzonych możliwości, używając uwierzytelniania bez kluczy.


## Instalacja wymaganych pakietów Python

Najpierw musimy zainstalować niezbędne pakiety Python:
- **azure-ai-projects**: Podstawowy SDK dla Azure AI Projects
- **azure-ai-agents**: SDK Azure AI Agents do tworzenia i zarządzania agentami
- **azure-identity**: Zapewnia bezkluczowe uwierzytelnianie za pomocą DefaultAzureCredential
- **mcp**: Implementacja Model Context Protocol dla Pythona


## Korzyści z uwierzytelniania bez kluczy

Ten notebook demonstruje **uwierzytelnianie bez kluczy**, które oferuje kilka zalet:
- ✅ **Brak konieczności zarządzania kluczami API** - Wykorzystuje uwierzytelnianie oparte na tożsamości Azure
- ✅ **Zwiększone bezpieczeństwo** - Brak przechowywania sekretów w kodzie lub plikach konfiguracyjnych
- ✅ **Automatyczna rotacja poświadczeń** - Azure zarządza cyklem życia poświadczeń
- ✅ **Kontrola dostępu oparta na rolach** - Wykorzystuje Azure RBAC do precyzyjnego zarządzania uprawnieniami
- ✅ **Wsparcie dla wielu środowisk** - Działa bezproblemowo zarówno w środowisku deweloperskim, jak i produkcyjnym

`DefaultAzureCredential` automatycznie wybiera najlepsze dostępne źródło poświadczeń:
1. **Managed Identity** (podczas uruchamiania w Azure)
2. Poświadczenia **Azure CLI** (w trakcie lokalnego rozwoju)
3. Poświadczenia **Visual Studio**
4. **Zmienne środowiskowe** (jeśli skonfigurowane)
5. Uwierzytelnianie przez **interaktywną przeglądarkę** (jako ostateczność)


## Konfiguracja uwierzytelniania bez kluczy

**Wymagania wstępne dla uwierzytelniania bez kluczy:**

### Dla lokalnego środowiska:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Dla środowisk Azure:
- Włącz **System-assigned Managed Identity** na swoim zasobie Azure
- Przypisz odpowiednie **role RBAC** do zarządzanej tożsamości:
  - `Cognitive Services OpenAI User` dla dostępu do Azure OpenAI
  - `AI Developer` dla dostępu do projektów Azure AI

### Zmienne środowiskowe (opcjonalne):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nie potrzebujesz kluczy API ani ciągów połączeń!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importuj Wymagane Biblioteki

Zaimportuj niezbędne moduły Pythona:
- **os, time**: Standardowe biblioteki Pythona do zmiennych środowiskowych i opóźnień
- **AIProjectClient**: Główny klient dla Azure AI Projects
- **DefaultAzureCredential**: Uwierzytelnianie bez kluczy dla usług Azure
- **Klasy związane z MCP**: Do tworzenia i zarządzania narzędziami MCP oraz obsługi zatwierdzeń


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfiguracja ustawień serwera MCP

Skonfiguruj ustawienia serwera MCP za pomocą zmiennych środowiskowych z domyślnymi wartościami zapasowymi:
- **MCP_SERVER_URL**: URL serwera MCP (domyślnie Microsoft Learn API)
- **MCP_SERVER_LABEL**: Etykieta identyfikująca serwer MCP (domyślnie "mslearn")

Takie podejście umożliwia elastyczną konfigurację w różnych środowiskach.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Utwórz klient projektu Azure AI (uwierzytelnianie bez klucza)

Zainicjalizuj klient projektu Azure AI, korzystając z **uwierzytelniania bez klucza**:
- **endpoint**: URL punktu końcowego projektu Azure AI Foundry
- **credential**: Używa `DefaultAzureCredential()` dla bezpiecznego uwierzytelniania bez klucza
- **Brak wymaganych kluczy API**: Automatycznie wykrywa i wykorzystuje najlepsze dostępne poświadczenia

**Proces uwierzytelniania:**
1. Sprawdza tożsamość zarządzaną (w środowiskach Azure)
2. W razie potrzeby korzysta z poświadczeń Azure CLI (dla lokalnego środowiska deweloperskiego)
3. Wykorzystuje inne dostępne źródła poświadczeń, jeśli to konieczne

Takie podejście eliminuje konieczność zarządzania kluczami API lub ciągami połączeń w kodzie.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Tworzenie definicji narzędzia MCP

Utwórz narzędzie MCP, które łączy się z serwerem Microsoft Learn MCP:
- **server_label**: Identyfikator serwera MCP
- **server_url**: Punkt końcowy URL serwera MCP
- **allowed_tools**: Opcjonalna lista ograniczająca, które narzędzia mogą być używane (pusta lista pozwala na użycie wszystkich narzędzi)

To narzędzie umożliwi agentowi dostęp do dokumentacji i zasobów Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Tworzenie Agenta i Prowadzenie Rozmowy (Praca Bez Kluczy)

Ten szczegółowy rozdział przedstawia kompletny **proces pracy agenta bez kluczy**:

1. **Tworzenie Agenta AI**: Skonfiguruj agenta z modelem GPT-4.1 nano i narzędziami MCP  
2. **Tworzenie Wątku**: Utwórz wątek rozmowy do komunikacji  
3. **Wysyłanie Wiadomości**: Zapytaj agenta o różnice między Azure OpenAI a OpenAI  
4. **Obsługa Zgód na Narzędzia**: Automatycznie zatwierdzaj wywołania narzędzi MCP, gdy jest to wymagane  
5. **Monitorowanie Wykonania**: Śledź postępy agenta i obsługuj wymagane działania  
6. **Wyświetlanie Wyników**: Pokaż szczegóły rozmowy i użycia narzędzi  

**Funkcje Bez Kluczy:**
- ✅ **Brak zakodowanych na stałe sekretów** - Cała autoryzacja obsługiwana przez tożsamość Azure  
- ✅ **Domyślne bezpieczeństwo** - Wykorzystuje kontrolę dostępu opartą na rolach  
- ✅ **Uproszczone wdrożenie** - Brak konieczności zarządzania poświadczeniami  
- ✅ **Przyjazne dla audytu** - Cały dostęp jest śledzony za pomocą tożsamości Azure  

Agent będzie korzystał z narzędzi MCP, aby uzyskać dostęp do zasobów Microsoft Learn z pełnym zabezpieczeniem i bez zarządzania kluczami API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
